In [6]:
pip install pandas

  Using cached pandas-1.2.2-cp38-cp38-macosx_10_9_x86_64.whl (10.5 MB)
  Using cached numpy-1.20.1-cp38-cp38-macosx_10_9_x86_64.whl (16.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import datetime
import warnings

warnings.filterwarnings('ignore')

In [2]:
from transform import *

# Players

In [3]:
matches_2019_path = "../data/raw/2019/rodada-"

In [4]:
def read_all_files(path, max_round):
    # return all dataframes concated
    df_rounds = []
    # We have match within rounds from 1 to max_round
    for round_number in range(1, max_round):
        filename = path + "{round_number}.csv".format(round_number=round_number)
        df_rounds.append(pd.read_csv(filename))
    return pd.concat(df_rounds)

In [5]:
df_rounds = read_all_files(matches_2019_path, 39)

In [6]:
df_rounds.shape

(30581, 33)

In [7]:
#Remove all players that didn't play the match
df_rounds = df_rounds.dropna(subset=['FS', 'RB', 'PE', 'FC', 'G', 'FF', 'FT', 'FD', 'DD', 'GS', 'SG', 'A', 'CA', 'I', 'CV', 'PP', 'GC', 'DP'],
                             how='all')

In [8]:
df_useful_columns = df_rounds[['atletas.atleta_id', 'atletas.clube_id', 'atletas.rodada_id', 'atletas.posicao_id', 'atletas.preco_num', 'atletas.pontos_num',
                               'atletas.variacao_num', 'atletas.media_num', 'A', 'G', 'FD', 'FF', 'FT', 'PE', 'PP', 'RB', 'DD', 'DP', 'GS']]

In [9]:
# renaming columns acording our documentations
columns_map = {
    "atletas.atleta_id": "id_player", "atletas.clube_id": "id_team", "atletas.rodada_id": "round",
    "atletas.posicao_id": "position", "atletas.preco_num": "cartola_price", "atletas.variacao_num": "cartola_price_update",
    "atletas.pontos_num": "cartola_score", "atletas.media_num": "cartola_score_mean", "A":"goal_assistance",
    "G": "scored_goals", "FD": "saved_kicks", "FF": "wrong_kicks", "FT": "crossbar_kicks",
    "PE": "wrong_passes", "PP": "missed_penalties", "RB": "stolen_ball",
    "DD": "difficult_gk_saves", "DP": "gk_penalty_saves", "GS": "conceded_goals"
}

df_players_final = df_useful_columns.rename(columns=columns_map)

In [10]:
df_players_final

,id_player,id_team,round,position,cartola_price,cartola_score,cartola_price_update,cartola_score_mean,goal_assistance,scored_goals,saved_kicks,wrong_kicks,crossbar_kicks,wrong_passes,missed_penalties,stolen_ball,difficult_gk_saves,gk_penalty_saves,conceded_goals
0,80196,267,1,mei,11.10,2.0,-2.90,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
6,78445,262,1,lat,8.74,0.6,-3.26,0.60,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN
7,80853,284,1,zag,12.49,8.0,2.49,8.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
9,38229,282,1,lat,12.77,9.4,3.77,9.40,NaN,1.0,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN
10,38910,282,1,ata,18.94,12.3,3.94,12.30,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,98822,282,38,lat,1.70,0.8,-0.27,-0.34,NaN,NaN,1.0,1.0,NaN,14.0,NaN,4.0,NaN,NaN,NaN
839,98832,284,38,mei,10.48,0.0,0.00,5.61,2.0,3.0,4.0,7.0,NaN,23.0,NaN,12.0,NaN,NaN,NaN
842,105341,290,38,zag,0.83,0.0,0.00,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
843,105584,264,38,lat,1.84,6.2,0.84,6.20,1.0,NaN,NaN,1.0,NaN,2.0,NaN,1.0,NaN,NaN,NaN


# Teams

In [11]:
team_matches_2019_path = "../data/raw/2019/2019_partidas.csv"

In [12]:
matches_2019_df = pd.read_csv(team_matches_2019_path)

In [13]:
matches_2019_df.head()

,date,home_team,away_team,home_score,away_score,round
0,2019-04-27,282,314,2,1,1
1,2019-04-27,315,285,2,0,1
2,2019-04-27,262,283,3,1,1
3,2019-04-27,276,263,2,0,1
4,2019-04-28,293,267,4,1,1


In [14]:
# Convert date columns type from string to date
matches_2019_df["date"] = matches_2019_df.apply(lambda row: datetime.datetime.strptime(row["date"], '%Y-%m-%d'),
                                                                            axis=1)

In [15]:
# Create 'week_day' column
matches_2019_df["week_day"] = matches_2019_df.apply(lambda row: row["date"].weekday(), axis=1)

In [16]:
# Create has_won column
def col_has_won(row, col_score_1, col_score_2):
    
    if row[col_score_1] > row[col_score_2]:
        return True
    return False
    
matches_2019_df["home_has_won"] = matches_2019_df.apply(lambda row: col_has_won(row, "home_score", "away_score"), axis=1)
matches_2019_df["away_has_won"] = matches_2019_df.apply(lambda row: col_has_won(row, "away_score", "home_score"), axis=1)

In [17]:
# Create draw column
def col_is_draw(row):
    
    if row["home_score"] == row["away_score"]:
        return True
    return False
    
matches_2019_df["was_draw"] = matches_2019_df.apply(lambda row: col_is_draw(row), axis=1)

In [18]:
# Create was_home_team column
matches_2019_df["home_was_home_team"] = True
matches_2019_df["away_was_home_team"] = False

In [19]:
home_teams = matches_2019_df[["home_team", "away_team", "round", "week_day", "date", "home_has_won", "was_draw",
                                            "home_score", "away_score", "home_was_home_team"]] \
                                            .rename(columns={"home_team": "id_team",
                                                             "away_team": "id_opponent_team",
                                                             "home_has_won": "has_won",
                                                             "home_score": "team_goals",
                                                             "away_score": "opponent_team_goals",
                                                             "home_was_home_team": "was_home_team"})


away_teams = matches_2019_df[["away_team", "home_team", "round", "week_day", "date", "away_has_won", "was_draw",
                                            "away_score", "home_score", "away_was_home_team"]] \
                                            .rename(columns={"away_team": "id_team",
                                                             "home_team": "id_opponent_team",
                                                             "away_has_won": "has_won",
                                                             "away_score": "team_goals",
                                                             "home_score": "opponent_team_goals",
                                                             "away_was_home_team": "was_home_team"})


df_teams = pd.concat([home_teams, away_teams])

In [20]:
df_teams = apply_match_points(df_teams)
df_teams_score = apply_championship_score(df_teams)
df_teams_final = calcule_championship_position(df_teams_score)

In [21]:
df_teams_final

,id_team,id_opponent_team,round,week_day,date,has_won,was_draw,team_goals,opponent_team_goals,was_home_team,match_points,championship_score,championship_position
0,282,314,1,5,2019-04-27,True,False,2,1,True,3,3,1.0
1,315,285,1,5,2019-04-27,True,False,2,0,True,3,3,4.0
2,262,283,1,5,2019-04-27,True,False,3,1,True,3,3,10.0
3,276,263,1,5,2019-04-27,True,False,2,0,True,3,3,9.0
4,293,267,1,6,2019-04-28,True,False,4,1,True,3,3,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,284,290,38,6,2019-12-08,False,False,2,3,False,0,65,4.0
376,282,285,38,6,2019-12-08,False,False,1,2,False,0,48,13.0
377,275,283,38,6,2019-12-08,True,False,2,0,False,3,74,3.0
378,262,277,38,6,2019-12-08,False,False,0,4,False,0,90,1.0


In [22]:
players_clean_path = "../data/clean/players/"

df_players_final.to_csv(players_clean_path + "2019.csv", index=False)

In [23]:
teams_clean_path = "../data/clean/teams/"
df_teams_final.to_csv(teams_clean_path + "2019.csv", index=False)